In [1]:
# -*- coding: utf-8 -*-
"""
TransactionAlertPro.py
Enhanced Baseline for 2025 Esun AI Challenge (Binary Classification)

Key upgrades over the sample:
1) Rich account-level feature engineering (amount stats, behavior, graph-like features, channel/currency/self-txn mix, time recency & activity)
2) Robust parsing for date/time and numeric values
3) Class-imbalance handling and threshold tuning via validation set (F1 optimization)
4) Strong but simple model (RandomForestClassifier) + reproducibility

"""

import os
import math
import warnings
from typing import Tuple, List, Dict

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_recall_curve, average_precision_score, roc_auc_score, classification_report

warnings.filterwarnings("ignore")


# -----------------------------
# Utilities
# -----------------------------
def _to_numeric(s):
    """Best-effort numeric conversion; returns float (NaN if fail)."""
    try:
        return pd.to_numeric(s, errors="coerce")
    except Exception:
        return np.nan


def _to_datetime(s):
    """Best-effort datetime parsing."""
    try:
        return pd.to_datetime(s, errors="coerce", infer_datetime_format=True)
    except Exception:
        return pd.NaT


def _safe_div(a, b):
    """
    Safe division that works for scalars, numpy arrays, and pandas Series/DataFrames.
    Returns 0.0 where b is NaN or 0.
    """
    # 如果有任一是 pandas 物件（Series/DataFrame），用向量化邏輯
    if isinstance(a, (pd.Series, pd.DataFrame)) or isinstance(b, (pd.Series, pd.DataFrame)):
        # 將 b 轉為與 a 可廣播的陣列/序列來做遮罩
        mask = pd.notna(b) & (b != 0)
        # 使用 np.where 做逐元素安全除法
        return np.where(mask, a / b, 0.0)
    # 否則視為標量
    else:
        return (a / b) if (b is not None and b != 0) else 0.0



def _most_common(values: pd.Series, topk: int = 5) -> Dict:
    """Return counts of top-k most frequent elements (as dict)."""
    cnt = values.value_counts()
    top = cnt.head(topk)
    return {f"top_{i+1}_val": top.index[i] if i < len(top) else np.nan for i in range(topk)}


def _map_is_esun(v):
    """
    Map acct_type to 1 (esun) / 0 (others).
    Per spec: 01: 玉山；02: 他行（可能為字串'01','1'等）
    """
    if pd.isna(v):
        return np.nan
    s = str(v).strip()
    return 1 if s in {"1", "01", "esun", "ESUN", "玉山"} else 0


def _ensure_int(x):
    try:
        return int(x)
    except Exception:
        return np.nan


# -----------------------------
# Data Loading
# -----------------------------
def load_csvs(dir_path: str):
    df_txn = pd.read_csv(os.path.join(dir_path, 'acct_transaction.csv'))
    df_alert = pd.read_csv(os.path.join(dir_path, 'acct_alert.csv'))
    df_test = pd.read_csv(os.path.join(dir_path, 'acct_predict.csv'))
    print("[OK] Loaded datasets.")
    return df_txn, df_alert, df_test


# -----------------------------
# Feature Engineering
# -----------------------------
def engineer_features(df_txn: pd.DataFrame) -> pd.DataFrame:
    """
    Build robust account-level features using both sender and receiver roles.
    """
    df = df_txn.copy()

    # --- Robust typing ---
    # amounts
    if 'txn_amt' in df.columns:
        df['txn_amt'] = df['txn_amt'].apply(_to_numeric)
    else:
        raise ValueError("Column 'txn_amt' not found in transaction data.")

    # date & time
    if 'txn_date' in df.columns:
        df['txn_date_dt'] = df['txn_date'].apply(_to_datetime)
    else:
        df['txn_date_dt'] = pd.NaT

    if 'txn_time' in df.columns:
        # try to parse as HHMMSS or HH:MM:SS
        def _to_hour(v):
            if pd.isna(v):
                return np.nan
            s = str(v).strip()
            # remove non-digits except ':'
            s2 = ''.join(ch for ch in s if (ch.isdigit() or ch == ':'))
            if ':' in s2:
                try:
                    hh = int(s2.split(':')[0])
                    return hh if 0 <= hh <= 23 else np.nan
                except Exception:
                    return np.nan
            # assume integer HHMMSS
            try:
                s2 = s2.zfill(6)
                hh = int(s2[:2])
                return hh if 0 <= hh <= 23 else np.nan
            except Exception:
                return np.nan

        df['hour'] = df['txn_time'].apply(_to_hour)
    else:
        df['hour'] = np.nan

    # flags / categories
    for col in ['from_acct_type', 'to_acct_type']:
        if col in df.columns:
            df[col] = df[col].apply(_map_is_esun)
        else:
            df[col] = np.nan

    if 'is_self_txn' in df.columns:
        df['is_self_txn_f'] = df['is_self_txn'].map(lambda x: 1 if str(x).strip().upper() == 'Y' else (0 if str(x).strip().upper() == 'N' else np.nan))
    else:
        df['is_self_txn_f'] = np.nan

    if 'channel_type' in df.columns:
        # normalize channel_type as string categories
        df['channel_type_norm'] = df['channel_type'].astype(str).str.strip().fillna('UNK')
    else:
        df['channel_type_norm'] = 'UNK'

    if 'currency_type' in df.columns:
        df['currency_type_norm'] = df['currency_type'].astype(str).str.strip().fillna('UNK')
    else:
        df['currency_type_norm'] = 'UNK'

    # counterparties
    if not set(['from_acct', 'to_acct']).issubset(df.columns):
        raise ValueError("from_acct and to_acct must exist in transaction data.")

    # --- per-account aggregates (sender side) ---
    g_from = df.groupby('from_acct')
    send_amt_sum = g_from['txn_amt'].sum().rename('send_amt_sum')
    send_amt_mean = g_from['txn_amt'].mean().rename('send_amt_mean')
    send_amt_std = g_from['txn_amt'].std(ddof=0).fillna(0).rename('send_amt_std')
    send_amt_max = g_from['txn_amt'].max().rename('send_amt_max')
    send_amt_min = g_from['txn_amt'].min().rename('send_amt_min')
    send_amt_median = g_from['txn_amt'].median().rename('send_amt_median')
    send_cnt = g_from.size().rename('send_cnt')

    # unique counterparties sent to, reciprocity later
    send_unique_ctp = g_from['to_acct'].nunique().rename('send_unique_ctp')

    # hours, channels, currency, self-txn ratio on sender side
    send_night = g_from['hour'].apply(lambda s: ((s>=22) | (s<=6)).sum()).rename('send_night_cnt')
    send_hour_mean = g_from['hour'].mean().rename('send_hour_mean')
    send_hour_std = g_from['hour'].std(ddof=0).fillna(0).rename('send_hour_std')

    send_self_cnt = g_from['is_self_txn_f'].sum(min_count=1).fillna(0).rename('send_self_cnt')
    send_self_ratio = (send_self_cnt / send_cnt.replace(0, np.nan)).fillna(0).rename('send_self_ratio')

    # channel one-hot proportions (top categories)
    top_channels = df['channel_type_norm'].value_counts().head(8).index.tolist()
    send_channel_props = []
    for ch in top_channels:
        colname = f'send_ch_{ch}_prop'
        send_channel_props.append(
            g_from.apply(lambda s, ch=ch: (s['channel_type_norm']==ch).mean()).rename(colname)
        )

    # currency stats
    send_curr_nunique = g_from['currency_type_norm'].nunique().rename('send_curr_nunique')
    send_curr_twd_ratio = g_from.apply(lambda s: (s['currency_type_norm']=='TWD').mean()).rename('send_curr_twd_ratio')

    # esun-to-others ratio when sending
    send_to_others_ratio = g_from['to_acct_type'].mean().rename('send_to_esun_ratio')  # 1 means esun, 0 others
    # we also keep 1 - ratio (others), but model can infer

    # recency & activity
    send_active_days = g_from['txn_date_dt'].nunique().rename('send_active_days')
    ref_date = df['txn_date_dt'].max()
    send_last_gap_days = g_from['txn_date_dt'].max().apply(lambda t: (ref_date - t).days if pd.notna(t) else np.nan).rename('send_last_gap_days')
    send_avg_amt_per_day = (send_amt_sum / send_active_days.replace(0, np.nan)).fillna(0).rename('send_avg_amt_per_day')

    # --- per-account aggregates (receiver side) ---
    g_to = df.groupby('to_acct')
    recv_amt_sum = g_to['txn_amt'].sum().rename('recv_amt_sum')
    recv_amt_mean = g_to['txn_amt'].mean().rename('recv_amt_mean')
    recv_amt_std = g_to['txn_amt'].std(ddof=0).fillna(0).rename('recv_amt_std')
    recv_amt_max = g_to['txn_amt'].max().rename('recv_amt_max')
    recv_amt_min = g_to['txn_amt'].min().rename('recv_amt_min')
    recv_amt_median = g_to['txn_amt'].median().rename('recv_amt_median')
    recv_cnt = g_to.size().rename('recv_cnt')
    recv_unique_ctp = g_to['from_acct'].nunique().rename('recv_unique_ctp')

    recv_night = g_to['hour'].apply(lambda s: ((s>=22) | (s<=6)).sum()).rename('recv_night_cnt')
    recv_hour_mean = g_to['hour'].mean().rename('recv_hour_mean')
    recv_hour_std = g_to['hour'].std(ddof=0).fillna(0).rename('recv_hour_std')

    recv_self_cnt = g_to['is_self_txn_f'].sum(min_count=1).fillna(0).rename('recv_self_cnt')
    recv_self_ratio = (recv_self_cnt / recv_cnt.replace(0, np.nan)).fillna(0).rename('recv_self_ratio')

    recv_channel_props = []
    for ch in top_channels:
        colname = f'recv_ch_{ch}_prop'
        recv_channel_props.append(
            g_to.apply(lambda s, ch=ch: (s['channel_type_norm']==ch).mean()).rename(colname)
        )

    recv_curr_nunique = g_to['currency_type_norm'].nunique().rename('recv_curr_nunique')
    recv_curr_twd_ratio = g_to.apply(lambda s: (s['currency_type_norm']=='TWD').mean()).rename('recv_curr_twd_ratio')

    recv_from_esun_ratio = g_to['from_acct_type'].mean().rename('recv_from_esun_ratio')

    recv_active_days = g_to['txn_date_dt'].nunique().rename('recv_active_days')
    recv_last_gap_days = g_to['txn_date_dt'].max().apply(lambda t: (ref_date - t).days if pd.notna(t) else np.nan).rename('recv_last_gap_days')
    recv_avg_amt_per_day = (recv_amt_sum / recv_active_days.replace(0, np.nan)).fillna(0).rename('recv_avg_amt_per_day')

    # --- graph-like reciprocity ---
    # For each account, proportion of counterparties with two-way flows
    # Build adjacency sets
    out_sets = df.groupby('from_acct')['to_acct'].apply(set)
    in_sets = df.groupby('to_acct')['from_acct'].apply(set)
    all_accts = set(out_sets.index).union(set(in_sets.index))

    reci_ratio = {}
    total_degree = {}
    bi_degree = {}

    for a in all_accts:
        outs = out_sets.get(a, set())
        ins = in_sets.get(a, set())
        deg = len(outs.union(ins))
        bi = len(outs.intersection(ins))
        total_degree[a] = deg
        bi_degree[a] = bi
        reci_ratio[a] = _safe_div(bi, deg)

    df_graph = pd.DataFrame({
        'acct': list(all_accts),
        'graph_degree': [total_degree[a] for a in all_accts],
        'graph_bi_degree': [bi_degree[a] for a in all_accts],
        'graph_reciprocity': [reci_ratio[a] for a in all_accts],
    })

    # --- merge all features to account level ---
    # Sender features -> index is 'from_acct'
    left = (
        pd.concat([
            send_amt_sum, send_amt_mean, send_amt_std, send_amt_max, send_amt_min, send_amt_median, send_cnt,
            send_unique_ctp, send_night, send_hour_mean, send_hour_std, send_self_cnt, send_self_ratio,
            send_curr_nunique, send_curr_twd_ratio, send_to_others_ratio, send_active_days,
            send_last_gap_days, send_avg_amt_per_day
        ] + send_channel_props, axis=1)
        .reset_index().rename(columns={'from_acct': 'acct'})
    )

    # Receiver features -> index is 'to_acct'
    right = (
        pd.concat([
            recv_amt_sum, recv_amt_mean, recv_amt_std, recv_amt_max, recv_amt_min, recv_amt_median, recv_cnt,
            recv_unique_ctp, recv_night, recv_hour_mean, recv_hour_std, recv_self_cnt, recv_self_ratio,
            recv_curr_nunique, recv_curr_twd_ratio, recv_from_esun_ratio, recv_active_days,
            recv_last_gap_days, recv_avg_amt_per_day
        ] + recv_channel_props, axis=1)
        .reset_index().rename(columns={'to_acct': 'acct'})
    )

    # Outer merge sender/receiver features
    feat = pd.merge(left, right, on='acct', how='outer')

    # Add simple totals / balances
    feat['total_amt_sum'] = feat['send_amt_sum'].fillna(0) + feat['recv_amt_sum'].fillna(0)
    feat['net_out_amt'] = feat['send_amt_sum'].fillna(0) - feat['recv_amt_sum'].fillna(0)
    feat['total_cnt'] = feat['send_cnt'].fillna(0) + feat['recv_cnt'].fillna(0)

    # Normalize some ratios
    feat['send_avg_amt'] = _safe_div(feat['send_amt_sum'].fillna(0), feat['send_cnt'].replace(0, np.nan))
    feat['recv_avg_amt'] = _safe_div(feat['recv_amt_sum'].fillna(0), feat['recv_cnt'].replace(0, np.nan))
    feat['send_unique_rate'] = _safe_div(feat['send_unique_ctp'].fillna(0), feat['send_cnt'].replace(0, np.nan))
    feat['recv_unique_rate'] = _safe_div(feat['recv_unique_ctp'].fillna(0), feat['recv_cnt'].replace(0, np.nan))

    # Merge graph features
    feat = feat.merge(df_graph, on='acct', how='left')

    # Determine account type (if account ever appears as from_acct or to_acct with acct_type)
    df_from_type = df[['from_acct', 'from_acct_type']].drop_duplicates().rename(columns={'from_acct': 'acct', 'from_acct_type': 'is_esun_from'})
    df_to_type = df[['to_acct', 'to_acct_type']].drop_duplicates().rename(columns={'to_acct': 'acct', 'to_acct_type': 'is_esun_to'})
    feat = feat.merge(df_from_type, on='acct', how='left').merge(df_to_type, on='acct', how='left')
    feat['is_esun'] = feat[['is_esun_from', 'is_esun_to']].max(axis=1)

    # Fill NaNs with 0 for model; keep acct id
    feat = feat.fillna(0)
    print(f"[OK] Feature engineering completed. Accounts: {len(feat)}; Features: {feat.shape[1]-1}")
    return feat


# -----------------------------
# Train / Validation / Test Split
# -----------------------------
def make_splits(feat_df: pd.DataFrame, df_alert: pd.DataFrame, df_test: pd.DataFrame):
    """
    Build training labels and test set.
    - Label for training: acct in df_alert['acct'] -> 1 else 0
    - Only use Esun accounts for training (per spec & to match test distribution)
    - Exclude test accounts from training
    """
    feat = feat_df.copy()

    # Labels
    alert_set = set(df_alert['acct'].astype(str))
    feat['label'] = feat['acct'].astype(str).isin(alert_set).astype(int)

    # Test mask
    test_set = set(df_test['acct'].astype(str))

    # Keep only esun in train
    train_df = feat[(~feat['acct'].astype(str).isin(test_set)) & (feat['is_esun'] == 1)].copy()
    X = train_df.drop(columns=['label'])
    y = train_df['label'].values

    # Test data: exactly test acct list joined with features (missing -> 0)
    test_feat = feat[feat['acct'].astype(str).isin(test_set)].copy()
    # Guarantee order same as df_test
    X_test = df_test[['acct']].merge(test_feat.drop(columns=['label']), on='acct', how='left').fillna(0)

    print(f"[OK] Split -> Train accounts: {len(X)} (pos={sum(y)}, neg={len(y)-sum(y)}); Test accounts: {len(X_test)}")
    return X, y, X_test


# -----------------------------
# Modeling
# -----------------------------
def fit_model_and_tune_threshold(X: pd.DataFrame, y: np.ndarray, random_state: int = 42):
    """
    Train RandomForest with reasonable defaults and tune decision threshold on a validation set
    to maximize F1 (handles imbalance more sensibly).
    """
    # Features to use: drop identifiers only
    drop_cols = ['acct']
    feat_cols = [c for c in X.columns if c not in drop_cols]

    X_train, X_val, y_train, y_val = train_test_split(
        X[feat_cols], y, test_size=0.2, random_state=random_state, stratify=y
    )

    clf = RandomForestClassifier(
        n_estimators=600,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        n_jobs=-1,
        class_weight='balanced_subsample',
        random_state=random_state,
    )
    clf.fit(X_train, y_train)

    # Validation metrics & threshold tuning
    val_proba = clf.predict_proba(X_val)[:, 1]
    ap = average_precision_score(y_val, val_proba)
    auc = roc_auc_score(y_val, val_proba)

    # pick threshold with best F1
    prec, rec, thr = precision_recall_curve(y_val, val_proba)
    f1s = 2 * prec * rec / (prec + rec + 1e-12)
    best_idx = np.nanargmax(f1s)
    best_thr = 0.5
    # precision_recall_curve returns len(thr)=len(prec)-1; align safely
    if best_idx < len(thr):
        best_thr = thr[best_idx]
    else:
        # edge case: choose 0.5 if cannot align
        best_thr = 0.5

    y_val_pred = (val_proba >= best_thr).astype(int)
    f1 = f1_score(y_val, y_val_pred)

    print(f"[VAL] AP={ap:.4f}  AUC={auc:.4f}  F1@best={f1:.4f}  thr={best_thr:.4f}")
    print("[VAL] Classification report:\n", classification_report(y_val, y_val_pred, digits=4))
    return clf, feat_cols, best_thr


def predict_test(clf, feat_cols: List[str], threshold: float, X_test: pd.DataFrame):
    proba = clf.predict_proba(X_test[feat_cols])[:, 1]
    y_pred = (proba >= threshold).astype(int)
    return y_pred, proba


# -----------------------------
# Output
# -----------------------------
def save_submission(path: str, df_test: pd.DataFrame, X_test: pd.DataFrame, y_pred: np.ndarray):
    df_pred = pd.DataFrame({
        'acct': X_test['acct'].values,
        'label': y_pred.astype(int)
    })
    out = df_test[['acct']].merge(df_pred, on='acct', how='left').fillna(0)
    out.to_csv(path, index=False, encoding='utf-8-sig')
    print(f"[OK] Saved submission to: {path}")


# -----------------------------
# Main
# -----------------------------
def main():
    # === change this to your data directory ===
    dir_path = "data"

    df_txn, df_alert, df_test = load_csvs(dir_path)
    feat_df = engineer_features(df_txn)
    X, y, X_test = make_splits(feat_df, df_alert, df_test)
    clf, feat_cols, thr = fit_model_and_tune_threshold(X, y, random_state=42)
    y_pred, _ = predict_test(clf, feat_cols, thr, X_test)
    save_submission("result.csv", df_test, X_test, y_pred)


if __name__ == "__main__":
    main()


[OK] Loaded datasets.
[OK] Feature engineering completed. Accounts: 1800106; Features: 67
[OK] Split -> Train accounts: 328988 (pos=1004, neg=327984); Test accounts: 4780
[VAL] AP=0.5364  AUC=0.9603  F1@best=0.5826  thr=0.1500
[VAL] Classification report:
               precision    recall  f1-score   support

           0     0.9985    0.9992    0.9989     65597
           1     0.6667    0.5174    0.5826       201

    accuracy                         0.9977     65798
   macro avg     0.8326    0.7583    0.7907     65798
weighted avg     0.9975    0.9977    0.9976     65798

[OK] Saved submission to: result.csv
